Oto moja kopia zadania numer siedem

In [ ]:
import os
import pandas as pd
import scraper
import translator
import sentimentAnalysis
import graphAnalysis
import utils
import importlib
importlib.reload(utils)
importlib.reload(scraper)
from nltk.corpus import wordnet
from itertools import chain
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.tokenize import word_tokenize

In [ ]:
#Wczytywanie danych - Data Load
oneonetwo = pd.read_csv("data"+os.sep+"part"+os.sep+"Suomi112_cpd.csv", encoding='utf-8-sig')
sos_live = pd.read_csv("data"+os.sep+"part"+os.sep+"SosLive_cpd.csv", encoding='utf-8-sig')
#Połączenie danych - Data Concat
data = pd.concat([oneonetwo,sos_live],ignore_index = True)

In [ ]:
#Lista Indykatorów - List of indicators
lista_indykatorow = ["perceived_ease_of_use", "perceived_usefulness", "satisfaction", "attitude", "behavioral_intentions"]
#Stworzyłem poszczególne listy słów kluczy: - I created lists of the keyword:
slowa_klucze  = \
{
    "perceived_ease_of_use" : ["simple","easy","intuitive","convenience","explainable","quick","speed","children","elderly"],
    "perceived_usefulness"  : ["efficient", "useful", "helpful", "work", "effective","revolutionary","implement"],
    "satisfaction"          : ["satisfaction","satisfy", "fulfill", "gratify", "meet", "beneficial", "content", "happy", "appeasement"],
    "attitude"              : [ "important", "emotion", "attitude", "opinion", "grade"],
    "behavioral_intentions" : ["usage","intentions", "interaction","plan","utilize","behavioral"],
}



In [ ]:
# Ta funkcja tworzy listę synonimóœ i nie tylku z listy słów - This function creates a list of synonyms and more from the list of words
def dobierz_synonimy_i_nie_tylko(slowa):
    slowa_klucze_zwrotne = []
    for slowo in slowa:
        slowa_klucze_zwrotne.append(slowo)
        #Synonims:
        synonimy = []
        for syn in wordnet.synsets(slowo):
            for s in syn.lemmas():
                synonimy.append(s.name())
        slowa_klucze_zwrotne.extend(synonimy)

        #Hypernims:
        parabole = []
        for par in wordnet.synsets(slowo):
            for p in par.hypernyms():
                for pl in p.lemmas():
                    parabole.append(pl.name())
        slowa_klucze_zwrotne.extend(parabole)

        zmiekczenia = []
        for zmi in wordnet.synsets(slowo):
            for zm in zmi.hyponyms():
                for z in zm.lemmas():
                    zmiekczenia.append(z.name())
        slowa_klucze_zwrotne.extend(zmiekczenia)

    return(set(slowa_klucze_zwrotne))

In [ ]:
#Wyszukiwanie synonimów:
slowa_klucze["perceived_ease_of_use"] = dobierz_synonimy_i_nie_tylko(slowa_klucze["perceived_ease_of_use"])
slowa_klucze["perceived_usefulness"] = dobierz_synonimy_i_nie_tylko(slowa_klucze["perceived_usefulness"] )
slowa_klucze["satisfaction"] = dobierz_synonimy_i_nie_tylko(slowa_klucze["satisfaction"] )
slowa_klucze["attitude"] = dobierz_synonimy_i_nie_tylko(slowa_klucze["attitude"])
slowa_klucze["behavioral_intentions"] = dobierz_synonimy_i_nie_tylko(slowa_klucze["behavioral_intentions"])

In [ ]:
#Można wypisać wyniczki - sanity check data presentation:
slowa_klucze["attitude"]

Preprocessing danych:

In [ ]:
import utils, importlib
importlib.reload(utils) # to keep the .py file up to date when coding
processor = utils.Processor() # Init preprocessing <- there is a path to set up where to dowloads this v
processor.ini_dowload() #dowload all the nessesary files to do the preprocesing (like the dictionary of the stopwords)

In [ ]:
from nltk import pos_tag
def preprocesing_danych(dane_dane_danedu_dane_danedu_uu):
    dane = dane_dane_danedu_dane_danedu_uu
    dane = processor.preprocess(str(dane))
    dane = processor.tokenize(dane)
    dane = processor.remove_stopwords(dane,remove_len=3) # Remove stopwords (like in the topic task)
    dane = processor.process_tokens(dane)  # Strip the words into the root words
    #dane = pos_tag(dane)
    return dane

In [ ]:
slowa_klucze["perceived_ease_of_use"]  = preprocesing_danych(slowa_klucze["perceived_ease_of_use"])
slowa_klucze["perceived_usefulness"]   = preprocesing_danych(slowa_klucze["perceived_usefulness"] )
slowa_klucze["satisfaction"]           = preprocesing_danych(slowa_klucze["satisfaction"]         )
slowa_klucze["attitude"]               = preprocesing_danych(slowa_klucze["attitude"]             )
slowa_klucze["behavioral_intentions"]  = preprocesing_danych(slowa_klucze["behavioral_intentions"])

In [ ]:
wszystkie_slowa_klucze = []
wszystkie_slowa_klucze.append(slowa_klucze["perceived_ease_of_use"])
wszystkie_slowa_klucze.append(slowa_klucze["perceived_usefulness"] )
wszystkie_slowa_klucze.append(slowa_klucze["satisfaction"] )
wszystkie_slowa_klucze.append(slowa_klucze["attitude"] )
wszystkie_slowa_klucze.append(slowa_klucze["behavioral_intentions"] )

In [ ]:
from gensim.corpora import Dictionary
keyword_dictionary = Dictionary(wszystkie_slowa_klucze)
keyword_bow_corpus = [keyword_dictionary.doc2bow(doc) for doc in wszystkie_slowa_klucze] #  dictionary reporting how many words and how many times those words appear
from gensim import corpora, models
keyword_tfidf = models.TfidfModel(keyword_bow_corpus)
corpus_tfidf = keyword_tfidf[keyword_bow_corpus]

In [ ]:
#Preproces the reviews:
data["preprocessed_reviews"] = data["content"].apply(lambda x: preprocesing_danych(x))

In [ ]:
#Vectorize the reviews:
from gensim.corpora import Dictionary
review_dictionary = Dictionary(data["preprocessed_reviews"])
review_bow_corpus = [review_dictionary.doc2bow(doc) for doc in data["preprocessed_reviews"]] #  dictionary reporting how many words and how many times those words appear
data_vectorized = keyword_tfidf[review_bow_corpus]
data["vectorized_reviews"] = data_vectorized.corpus

In [ ]:
#Classification:
from thefuzz import fuzz
from thefuzz import process
import numpy as np
def classify(rewiew,all_keywords,threshold=10):
    rewiew = " ".join(rewiew)
    scores = []
    for keywords in all_keywords:
        keywords = " ".join(keywords)
        scores.append(fuzz.token_set_ratio(rewiew,keywords))
    if max(scores) < threshold: #scores are from 0 to 100
        return -1
    best = np.argmax(scores) #ID of the best keyword
    return best

data["class"] = data["preprocessed_reviews"].apply(lambda x : classify(x, wszystkie_slowa_klucze))

In [ ]:
data["class"]

In [ ]:
# Create five horses of the TAM indicators
horse_ease_of_use     =    data.drop(data[data["class"] != 0].index)
horse_usefulness      =    data.drop(data[data["class"] != 1].index)
horse_satisfaction    =    data.drop(data[data["class"] != 2].index)
horse_attitude        =    data.drop(data[data["class"] != 3].index)
horse_behavioral      =    data.drop(data[data["class"] != 4].index)